In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn import metrics 
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

import keras
from scikeras.wrappers import KerasClassifier
from keras.models import Sequential 
from keras.layers import Dense
from keras.optimizers import SGD


In [ ]:
path=r'/home/ludovico/ML-project/data/monks-2'

#importing training and test set
monk2_train = pd.read_csv(path+'.train', header=None, delim_whitespace=True, dtype=str)
monk2_test = pd.read_csv(path+'.test', header=None, delim_whitespace=True, dtype=str)

#separating input from target
y_train=monk2_train[0]
x_train=monk2_train[monk2_train.columns[1:-1]]
y_test=monk2_test[0]
x_test=monk2_test[monk2_train.columns[1:-1]]

# encoded data
encoder = OneHotEncoder(sparse_output=False)
x_train = encoder.fit_transform(x_train)
x_test = encoder.fit_transform(x_test)

## Creating the network using Keras 
We add 1 hidden layer with 17 (x_train.shape[1]) number of input units, dens_nparams number of units in the hidden layer, SGD as optimizer algorithm and 1 output unit with sigmoid activation function. 

In [ ]:
def create_model(init='uniform', activation='relu', nbr_features=17, dense_nparams=10,lr=0.01,momentum=0.1,w_d=0.0001, nest=False):
    model = Sequential()
    model.add(Dense(dense_nparams, activation, input_shape=(nbr_features,), kernel_initializer=init,)) 
    model.add(Dense(1, activation='sigmoid',kernel_initializer=init))
    model.compile(loss='binary_crossentropy', optimizer=SGD(lr,momentum,nest,w_d),metrics=["accuracy"])
    return model

# callback to use earlystopping 
callback = keras.callbacks.EarlyStopping(monitor='loss',patience=3)

kears_estimator = KerasClassifier(model=create_model,verbose=0)

## Model selection

In [ ]:
keras.utils.set_random_seed(42)
param_grid = {
    'epochs': [600],
    'model__dense_nparams': [5],
    'model__init': [ 'normal'], 
    'batch_size':[50],
    'model__lr':[0.1],
    'model__momentum':[0.4],
    'model__w_d':[0.001],
    'model__activation':['relu']    
}

grid_search = GridSearchCV(estimator=kears_estimator,  
                    n_jobs=-1, 
                    verbose=3,
                    return_train_score=True,
                    scoring='accuracy',
                    cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=0),
                    param_grid=param_grid,
                    )

MLP=grid_search.fit(x_train, y_train)

### Evaluating the MLP classifier 

In [ ]:
cv_results_df = pd.DataFrame(MLP.cv_results_)
best_model_index=MLP.best_index_

print('best params',MLP.best_params_) 

val_loss=cv_results_df['mean_test_score'][best_model_index]
val_std=cv_results_df['std_test_score'][best_model_index]
train_loss=cv_results_df['mean_train_score'][best_model_index]
train_std=cv_results_df['std_train_score'][best_model_index]
print('Train acc:',train_loss,'+/-', train_std)
print('Validation acc:',val_loss,'+/-', val_std)

cv_results_df

## Learning curve

In [ ]:
keras.utils.set_random_seed(42)
x_train_cl, x_val, y_train_cl, y_val = train_test_split(x_train, y_train, test_size=0.20, random_state=0, stratify=y_train, shuffle=True)

model=KerasClassifier(model=create_model,**grid_search.best_params_)
history = model.fit(x_train_cl, y_train_cl.astype(np.float32),validation_data=(x_val,y_val.astype(np.float32)))

# list all data in history

plt.rcParams.update({'font.size': 12})

print(history.history_.keys())
# summarize history for accuracy
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history_['accuracy'])
plt.plot(history.history_['val_accuracy'], linestyle='--')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='best')
plt.subplot(1,2,2)
# summarize history for loss
plt.plot(history.history_['loss'])
plt.plot(history.history_['val_loss'], linestyle='--')

plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='best')
plt.show()

## Model assessment

In [ ]:
y_pred =MLP.predict(x_test)

print('Accuracy Score -  Test-error:', metrics.accuracy_score(y_test, y_pred))  

print('\n Classification report',metrics.classification_report(y_test, y_pred))

cm = metrics.confusion_matrix(y_test, y_pred, labels=MLP.classes_)
disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm,
                               display_labels=MLP.classes_)
disp.plot()
plt.show()